# RFM Analysis

We have been asked to conduct an RFM analysis on the customer from the United Kingdom for online_retail.

RFM refers to Recency, Frequency, Monetary. Thus, we will need to calculate for each customer ID:

Recency - The time since the customers last transaction
Frequency - How many transactions the customer has made
Monetaty - How much has the customer spent 

We will rank each customer from 1 - 5 for each metric. Those customers with a rank of 5 in each category, or 555, will be Champions.

In [20]:
###############################################################
# Loading The Data
###############################################################

import pandas as pd
import datetime as dt

df_ = pd.read_excel("online_retail.xlsx", sheet_name = "Year 2009-2010")

In [62]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [63]:
###############################################################
# Exploratory Data Analysis
###############################################################

df.Country.value_counts()

United Kingdom          485852
EIRE                      9670
Germany                   8129
France                    5772
Netherlands               2769
Spain                     1278
Switzerland               1187
Portugal                  1101
Belgium                   1054
Channel Islands            906
Sweden                     902
Italy                      731
Australia                  654
Cyprus                     554
Austria                    537
Greece                     517
United Arab Emirates       432
Denmark                    428
Norway                     369
Finland                    354
Unspecified                310
USA                        244
Japan                      224
Poland                     194
Malta                      172
Lithuania                  154
Singapore                  117
RSA                        111
Bahrain                    107
Canada                      77
Hong Kong                   76
Thailand                    76
Israel  

In [64]:
df = df[df["Country"]=="United Kingdom"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [65]:
# Shape is equal to the value counts for United Kingdom previously 

df.shape

(485852, 8)

In [66]:
# Null values

df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    106429
Country             0
dtype: int64

In [67]:
# The number of unique products

df['Description'].nunique()

4657

In [68]:
# How many of each product do you have?

df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3433
REGENCY CAKESTAND 3 TIER              2005
STRAWBERRY CERAMIC TRINKET BOX        1730
ASSORTED COLOUR BIRD ORNAMENT         1381
HOME BUILDING BLOCK WORD              1347
Name: Description, dtype: int64

In [69]:
# Which the most ordered product

df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,52252
WORLD WAR 2 GLIDERS ASSTD DESIGNS,51434
BROCADE RING PURSE,47495
ASSORTED COLOUR BIRD ORNAMENT,41855
PACK OF 72 RETRO SPOT CAKE CASES,36393


In [70]:
df["Invoice"].nunique()

26633

In [71]:
# Those invoices with a C preceeding them have a negative quantity. 

df[df["Invoice"].str.contains("C", na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
196,C489459,90200A,PURPLE SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom
197,C489459,90200D,PINK SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom
198,C489459,90200B,BLACK SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom
199,C489459,90200E,GREEN SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom
200,C489459,90200C,BLUE SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom


In [72]:
df = df[~df["Invoice"].str.contains("C", na=False)]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [73]:
# Add a total price column
df["TotalPrice"] = df["Quantity"] * df["Price"]

# What are the most expensive products
df.sort_values("TotalPrice", ascending=False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.09,NaN,United Kingdom,25111.09
432176,530715,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,9360,2010-11-04 11:36:00,1.69,15838.0,United Kingdom,15818.40
517955,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom,13541.33
135015,502265,M,Manual,1,2010-03-23 15:28:00,10953.50,NaN,United Kingdom,10953.50
135013,502263,M,Manual,1,2010-03-23 15:22:00,10953.50,12918.0,United Kingdom,10953.50


In [74]:
###############################################################
# Data Preparation
###############################################################

df.dropna(inplace=True)

In [75]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,370951.0,11.994444,67.394793,1.0,2.00,4.00,12.00,10000.0
Price,370951.0,3.145220,30.551482,0.0,1.25,1.95,3.75,10953.5
Customer ID,370951.0,15565.407693,1593.187209,12346.0,14215.00,15601.00,16940.00,18287.0
TotalPrice,370951.0,19.988505,73.930472,0.0,4.25,10.50,17.85,15818.4


In [82]:
###############################################################
# Calculating RFM Metrics
###############################################################

# Recency, Frequency, Monetary

print(df["InvoiceDate"].max())
today_date = dt.datetime(2010, 12, 11)
print(today_date)


rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda date: (today_date - date.max()).days, 
                                     "Invoice": lambda num : len(num),
                                     "TotalPrice": lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ["recency", "frequency", "monetary"]

rfm = rfm[(rfm["monetary"]) > 0 & (rfm['frequency'] > 0)]


rfm.head()

2010-12-09 20:01:00
2010-12-11 00:00:00


,recency,frequency,monetary
Customer ID,,,
12346.0,165,33,372.86
12608.0,40,16,415.79
12745.0,122,22,723.85
12746.0,176,17,254.55
12747.0,5,154,5080.53


In [86]:
###############################################################
# Calculating The RFM Scores
###############################################################

# Recency Score
rfm["RecencyScore"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])


# Frequency Score
rfm["FrequencyScore"] = pd.qcut(rfm['frequency'], 5, labels=[5, 4, 3, 2, 1])


# Monetary Score
rfm["MonetaryScore"] = pd.qcut(rfm["monetary"], 5, labels=[5, 4, 3, 2, 1])


# RFM_Score
rfm["RFM_Score"] = (rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str) + rfm['MonetaryScore'].astype(str))

# Champions
rfm[rfm["RFM_Score"]=="555"].head()





,recency,frequency,monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score
Customer ID,,,,,,,
13011.0,8,3,50.55,5,5,5,555
13254.0,12,4,51.41,5,5,5,555
13747.0,9,5,169.80,5,5,5,555
14119.0,11,11,183.37,5,5,5,555
14491.0,8,4,127.20,5,5,5,555


In [87]:
# Count Per Customer Category
rfm["RFM_Score"].value_counts()

511    275
155    230
411    148
144    129
255    119
      ... 
341      2
225      2
125      1
525      1
114      1
Name: RFM_Score, Length: 118, dtype: int64

In [88]:
# Need Attention
rfm[rfm["RFM_Score"]=="111"].head()

,recency,frequency,monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score
Customer ID,,,,,,,
12957.0,177,252,4689.43,1,1,1,111
14685.0,212,195,4619.79,1,1,1,111
16335.0,187,366,2774.00,1,1,1,111
17032.0,183,239,3004.60,1,1,1,111
17113.0,210,159,2322.59,1,1,1,111


In [89]:
###############################################################
# Naming & Analysing RFM Segments
###############################################################


# For this analysis we will only be considering recency and frequency not monetary

seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

In [90]:
rfm['Segment'] = rfm["RecencyScore"].astype(str) + rfm["FrequencyScore"].astype(str)
rfm.head()

,recency,frequency,monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Segment
Customer ID,,,,,,,,
12346.0,165,33,372.86,2,3,4,234,23
12608.0,40,16,415.79,3,4,4,344,34
12745.0,122,22,723.85,2,4,3,243,24
12746.0,176,17,254.55,2,4,4,244,24
12747.0,5,154,5080.53,5,1,1,511,51


In [93]:
# Replace the segment number by its corresponding segment name

rfm["Segment"] = rfm["Segment"].replace(seg_map, regex=True)
rfm.head()

,recency,frequency,monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Segment
Customer ID,,,,,,,,
12346.0,165,33,372.86,2,3,4,234,At_Risk
12608.0,40,16,415.79,3,4,4,344,Loyal_Customers
12745.0,122,22,723.85,2,4,3,243,At_Risk
12746.0,176,17,254.55,2,4,4,244,At_Risk
12747.0,5,154,5080.53,5,1,1,511,New_Customers


In [92]:
rfm.head()

,recency,frequency,monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Segment
Customer ID,,,,,,,,
12346.0,165,33,372.86,2,3,4,234,At_Risk
12608.0,40,16,415.79,3,4,4,344,Loyal_Customers
12745.0,122,22,723.85,2,4,3,243,At_Risk
12746.0,176,17,254.55,2,4,4,244,At_Risk
12747.0,5,154,5080.53,5,1,1,511,New_Customers


In [95]:
rfm_analysis = rfm[["Segment", "recency", "frequency", "monetary"]].groupby("Segment").agg(["mean", "count"])
rfm_analysis

recency         frequency           monetary      
                           mean count        mean count         mean count
Segment                                                                   
About_to_Sleep        53.255663   309  140.650485   309  2159.886832   309
At_Risk              187.306250   800   31.846250   800   679.125328   800
Cant_Loose           212.494024   502    7.280876   502   320.783745   502
Champions              8.359375   128   16.734375   128   562.854297   128
Hibernating          138.221429   280  130.503571   280  1926.509150   280
Loyal_Customers       41.148976   537   16.089385   537   476.794786   537
Need_Attention        52.510204   147   44.795918   147   910.731565   147
New_Customers          6.640884   362  366.883978   362  8146.353240   362
Potential_Loyalists   16.279412   680   72.805882   680  1785.031372   680
Promising             24.660714   224  277.357143   224  3923.356125   224

In [103]:
# Need Attetion Customers Will Recieve A Marketing Campaign Discount
need_attention = rfm[rfm["Segment"] == "Need_Attention"]
need_attention.head()

,recency,frequency,monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Segment
Customer ID,,,,,,,,
12903.0,64,41,833.53,3,3,3,333,Need_Attention
12987.0,56,38,753.51,3,3,3,333,Need_Attention
12989.0,60,33,6627.64,3,3,1,331,Need_Attention
13082.0,60,56,3722.12,3,3,1,331,Need_Attention
13195.0,50,52,172.38,3,3,5,335,Need_Attention
